In [1]:
from tlspt.datamodules.components.numpy_dataset import NumpyDataset
from tlspt.datamodules.components.base_site import BaseSiteDataset
from tlspt.datamodules.components.octree_dataset import OctreeDataset
from tlspt.datamodules.components.merged_dataset import MergedOctreeDataset
from tlspt.transforms import TLSSampler
from tlspt.models.pointmae.pointmae import PointMAE
from torch.utils.data import DataLoader
import torch
from matplotlib import pyplot as plt

import numpy as np

In [45]:
merged_dataset = MergedOctreeDataset(split_files=['/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv',
                                                  '/home/matt/work/tlsPT/data/plot_octrees/allen-poland/octrees/allen-poland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv',
                                                  '/home/matt/work/tlsPT/data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv'],
                                    split='train',
                                    scales=[1.5,1.5,1.5],
                                    feature_names=None,
                                    normalize=True,
                                    transform=TLSSampler(uniform_points=8192, farthest_points=1024)
                                    )

merged_dataloader = DataLoader(merged_dataset, batch_size=24, shuffle=True)

2024-11-18 14:14:15.302 | INFO     | tlspt.datamodules.components.base_site:__init__:49 - OctreeDataset(/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): reading splits from /home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv
2024-11-18 14:14:15.304 | INFO     | tlspt.datamodules.components.base_site:__init__:56 - OctreeDataset(/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): looking for 12 folders in /home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/
2024-11-18 14:14:15.305 | INFO     | tlspt.datamodules.components.base_site:__init__:64 - OctreeDataset(/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): found 12 plots for 'train' out of 12 plots defined in split f

In [46]:
model = PointMAE()

In [47]:
batch = next(iter(merged_dataloader))

In [48]:
type(batch)

dict

In [52]:
batch['points'].shape

torch.Size([24, 1024, 3])

In [53]:
test_out = model(batch)

In [54]:
test_out

tensor(nan, grad_fn=<DivBackward0>)

In [55]:
patches, centers = model.group(batch['points'])

In [56]:
patches.shape

torch.Size([24, 64, 32, 3])

In [57]:
centers.shape

torch.Size([24, 64, 3])

In [58]:
x_vis, mask, vis_pos_embeddings = model.forward_encoder(
    patches, centers
)

In [59]:
x_vis.shape

torch.Size([24, 26, 384])

In [60]:
mask.shape

torch.Size([24, 64])

In [61]:
vis_pos_embeddings.shape

torch.Size([24, 26, 384])

In [62]:
from tlspt.models.utils import get_masked, get_unmasked
masked_centers = get_masked(centers, mask)
masked_pos_embeddings = model.pos_encoder(masked_centers)

In [63]:
masked_centers.shape

torch.Size([24, 38, 3])

In [64]:
masked_pos_embeddings.shape

torch.Size([24, 38, 384])

In [65]:
B, N, _ = masked_pos_embeddings.shape
mask_tokens = model.mask_token.expand(B, N, -1)

In [66]:
mask_tokens = model.mask_token.expand(B, N, -1)

In [67]:
x_full = torch.cat((x_vis, mask_tokens), dim=1)

In [68]:
x_full.shape

torch.Size([24, 64, 384])

In [69]:
full_pos_embeddings = torch.cat((vis_pos_embeddings, masked_pos_embeddings), dim=1)

In [70]:
full_pos_embeddings.shape

torch.Size([24, 64, 384])

In [71]:
x_hat = model.forward_decoder(x_full, full_pos_embeddings, N)

In [72]:
x_hat.shape

torch.Size([24, 38, 32, 3])

In [73]:
x_gt = get_masked(patches, mask)

In [74]:
x_gt.shape

torch.Size([24, 38, 32, 3])

In [ ]:
loss = model.get_loss(x_hat, x_gt)

In [76]:
loss

tensor(nan, grad_fn=<DivBackward0>)

In [87]:
centers

tensor([[[-0.1478,  0.4641,  0.3917],
         [ 0.0641, -0.9596,  0.2205],
         [ 0.0122,  0.5055, -1.0000],
         ...,
         [-0.0491,  0.4716, -0.0163],
         [-0.0071,  0.4653,  0.1809],
         [ 0.1134, -0.7119, -0.0419]],

        [[ 0.0363,  0.0727,  0.2009],
         [ 0.4951,  1.0000, -0.0785],
         [-0.1115, -0.1128, -0.8347],
         ...,
         [ 0.0414, -0.0391, -0.0302],
         [-0.1236,  0.0942, -0.2071],
         [-0.1453, -0.0621, -0.1525]],

        [[ 0.6836, -0.6872,  0.5956],
         [-0.8212,  0.8112, -0.4206],
         [-0.6750, -0.9088, -0.6822],
         ...,
         [ 0.6055,  0.8566,  0.8072],
         [ 0.0652, -0.6491, -0.6295],
         [-0.3535, -0.5592,  0.5102]],

        ...,

        [[ 0.0568,  0.1720,  0.4719],
         [-0.8035,  0.2678, -0.7246],
         [ 0.3638,  0.8879, -0.6585],
         ...,
         [ 0.2496,  0.0987, -0.0933],
         [ 0.3226,  0.3526,  0.4978],
         [ 0.1122,  0.0603, -0.3591]],

        [[